In [114]:
import json

In [115]:
import mysql.connector
import json
import pymysql

In [116]:


# Establishing a connection to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="layakishore",
    database ="USERS_DATA")


In [117]:
if conn.is_connected():
    print("Connection to MySQL server is working.")
else:
    print("Connection to MySQL server is not established.")
mycur= conn.cursor()

Connection to MySQL server is working.


In [118]:
database_name = "USERS_DATA"

try:
   # mycur.execute(f"CREATE DATABASE {database_name}")
    print(f"Database '{database_name}' created successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")

Database 'USERS_DATA' created successfully


In [140]:
create_table_query = """
CREATE TABLE IF NOT EXISTS users1 (
    id VARCHAR(20) PRIMARY KEY,
    name VARCHAR(255),
    screen_name VARCHAR(255),
    location VARCHAR(255),
    verified BOOLEAN,
    followers_count INT,
    statuses_count INT,
    friends_count INT
)
"""

# Execute the query to create the table
#mycur.execute(create_table_query)

In [144]:


mycur.execute("describe Users1")
for X in mycur:
    print(X)

('id', 'varchar(20)', 'NO', 'PRI', None, '')
('name', 'varchar(255)', 'YES', '', None, '')
('screen_name', 'varchar(255)', 'YES', '', None, '')
('location', 'varchar(255)', 'YES', '', None, '')
('verified', 'tinyint(1)', 'YES', '', None, '')
('followers_count', 'int', 'YES', '', None, '')
('statuses_count', 'int', 'YES', '', None, '')
('friends_count', 'int', 'YES', '', None, '')


In [122]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='layakishore',
                             database='USERS_DATA',
                             cursorclass=pymysql.cursors.DictCursor)

In [146]:
insert_sql = "INSERT INTO Users1 (id, name, screen_name, location, verified, followers_count, statuses_count, friends_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

In [124]:
from elasticsearch import Elasticsearch

# Define Elasticsearch Cloud credentials
cloud_id = "finalproject:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ0NTI4OTc0ZTg5MmU0ZmY2YmU3NDNmODUzYTI1MDhjOSQyODdmZTVmMTc0MmM0ZWNmODgzN2M1YTNmNGE5MWQ1ZA=="
cloud_auth = ("elastic", "SvXQmBxTZWeUQC88AloilfrY")  # Credentials for accessing Elasticsearch Cloud

# Create Elasticsearch client with the specified parameters
es = Elasticsearch(cloud_id=cloud_id, basic_auth=cloud_auth)

# Check if the connection is successful
if es.ping():
    print("Connected to Elasticsearch Cloud")
else:
    print("Connection failed")



Connected to Elasticsearch Cloud


In [147]:
index_name = "tweeeets9_temp2"



# Define the mapping
mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 2000
    },
    "mappings": {
        "properties": {
            "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
            "id": {"type": "long"},
            "id_str": {"type": "keyword"},
            "text": {"type": "text", "fielddata": True},
            "user": {
                "properties": {
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "name": {"type": "text"},
                    "screen_name": {"type": "keyword"},
                  #  "dynamic": "false"
                }
            },
            "retweeted_status": {
                "properties": {
                    "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "text": {"type": "text", "fielddata": True},
                    "user": {
                        "properties": {
                            "id": {"type": "long"},
                            "id_str": {"type": "keyword"},
                            "name": {"type": "text"},
                            "screen_name": {"type": "keyword"}
                        }
                    },
                    "retweet_count": {"type": "integer"},
                    "favorite_count": {"type": "integer"},
                    "favorited": {"type": "boolean"},
                    "retweeted": {"type": "boolean"},
                    "possibly_sensitive": {"type": "boolean"}
                }
            },
            "retweet_count": {"type": "integer"},
            "favorite_count": {"type": "integer"},
            "favorited": {"type": "boolean"},
            "retweeted": {"type": "boolean"},
            "possibly_sensitive": {"type": "boolean"}
        }
    }
}

# Create the index with the mapping
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping, ignore=400)  # ignore 400 already exists code


/var/folders/57/144rtwxx2yq5tcjbw4w_dkcw0000gn/T/ipykernel_9002/3929832798.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=mapping, ignore=400)  # ignore 400 already exists code


In [134]:
def preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet):
    prepared_tweet = {
                "created_at": tweet.get("created_at"),
                "id": tweet.get("id"),
                "id_str": tweet.get("id_str"),
                "text": tweet.get("text"),
                "user": {
                    "id": user_info.get("id"),
                    "id_str": user_info.get("id_str"),
                    "name": user_info.get("name"),
                    "screen_name": user_info.get("screen_name")
                },
                "isretweet":isretweet,
                "retweeted_status": {
                    "created_at": retweeted_status.get("created_at"),
                    "id": retweeted_status.get("id"),
                    "id_str": retweeted_status.get("id_str"),
                    "text": retweeted_status.get("text"),
                    "user": {
                        "id": retweeted_user_info.get("id"),
                        "id_str": retweeted_user_info.get("id_str"),
                        "name": retweeted_user_info.get("name"),
                        "screen_name": retweeted_user_info.get("screen_name")
                    },
                    "retweet_count": retweeted_status.get("retweet_count"),
                    "favorite_count": retweeted_status.get("favorite_count"),
                    "favorited": retweeted_status.get("favorited"),
                    "retweeted": retweeted_status.get("retweeted"),
                    "possibly_sensitive": retweeted_status.get("possibly_sensitive")
                } if retweeted_status else {},
                "retweet_count": tweet.get("retweet_count"),
                "favorite_count": tweet.get("favorite_count"),
                "favorited": tweet.get("favorited"),
                "retweeted": tweet.get("retweeted"),
                "possibly_sensitive": tweet.get("possibly_sensitive")
            }
    
    return prepared_tweet

In [148]:
index_body_tweetids = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 1000,  # Limiting the number of fields
    },
    "mappings": {
        "properties": {
            "tweet_id": {"type": "keyword"},  # Using keyword for exact matches
        }
    }
}

# Create the index
index_name_tweetids = "tweet_idss2_temp2"
if not es.indices.exists(index=index_name_tweetids):
    es.indices.create(index=index_name_tweetids, body=index_body_tweetids)
    print(f"Index '{index_name_tweetids}' created.")
else:
    print(f"Index '{index_name_tweetids}' already exists.")

Index 'tweet_idss2_temp2' created.


In [149]:
index_body_userids = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 1000,  # Limiting the number of fields
    },
    "mappings": {
        "properties": {
            "user_id": {"type": "keyword"},  # Using keyword for exact matches
        }
    }
}

# Create the index
index_name_userids = "user_idsss2_temp2"
if not es.indices.exists(index=index_name_userids):
    es.indices.create(index=index_name_userids, body=index_body_userids)
    print(f"Index '{index_name_userids}' created.")
else:
    print(f"Index '{index_name_userids}' already exists.")

Index 'user_idsss2_temp2' created.


new index for retweets by this user
new index for likes by this user
new index for recent searches by this user
new index for search?
Relavance Order
Total runtime of the program is 750.3318951129913 seconds

search by string, hashtag,  and user at the minimum. Allow for a time range in these searches. You will also need some notion of relevance to rank the search results. 

Show the meta-data for each tweet such as (a) the author of the tweet,  (b) when it was tweeted, (c) how many times it was retweeted. If the user of the search application clicks on the number of times it was retweeted, show the information about the retweets (who retweeted, when etc.).

If user of the search application clicks on the author, show other tweets by that author or retweets by that author (if available). 

 How you store your information will impact the response times of your query. You must discuss this (in your presentation and report). 

In [150]:
def index_tweet(tweet):
    res = es.index(index=index_name, id=tweet["id"], document=tweet)
    #print(f"Indexed Tweet ID: {tweet['id']} Status: {res['result']}")


In [151]:
def insert_user_mysql(user):
    cursor.execute(insert_sql, (
                    user['id_str'],
                    user['name'],
                    user['screen_name'],
                    user['location'],
                    user['verified'],
                    user['followers_count'],
                    user['statuses_count'],
                    user['friends_count']
                ))
                # Commit the transaction
    print("insert successfull")
    return 

In [152]:
import time
start_time = time.time()
# Read the file and index each tweet

with connection.cursor() as cursor:
    with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2 copy", 'r') as file:
        for line in file:

            if not line.strip():
                continue

            tweet = json.loads(line.strip())
            tweet_id = tweet.get("id_str")

            if es.exists(index=index_name_tweetids, id=tweet_id):
                    print(f"Tweet ID {tweet_id} already exists.")
                    continue

            es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id})

            user_info = tweet.get("user", {})
            retweeted_status = tweet.get("retweeted_status", {})
            retweeted_user_info = retweeted_status.get("user", {}) if retweeted_status else {}



            if ( tweet['text'].startswith('RT') ):

                #check if user is there or not
                user_id=user_info.get("id_str")
                if es.exists(index=index_name_userids, id=user_id):
                    print(f"User ID {user_id} already exists.")
                else:
                    es.index(index=index_name_userids, id=user_id, document={"user_id": user_id})
                    insert_user_mysql(user_info)
                    connection.commit()
                    print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql
                    
                isretweet=True
                prepared_tweet=preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet)
                index_tweet(prepared_tweet)
                
                #check if Retweetuser is there or not
                if retweeted_status:
                   
                    tweet_id = retweeted_status.get("id_str")
                    if es.exists(index=index_name_tweetids, id=tweet_id):
                        print(f"Tweet ID {tweet_id} already exists.")
                        continue
                    else:
                        Ruser_id=retweeted_user_info.get("id_str")
                        if es.exists(index=index_name_userids, id=Ruser_id):
                            print(f"User ID {Ruser_id} already exists.")
                        else:
                            es.index(index=index_name_userids, id=user_id, document={"user_id": Ruser_id})
                            insert_user_mysql(retweeted_user_info)
                            connection.commit()
                            print("commit success")
                        #create a new original tweet for the retweet
                        isretweet=False
                        prepared_tweet=preparedTweet(tweet=retweeted_status,user_info=retweeted_user_info,retweeted_status={},retweeted_user_info={},isretweet=isretweet)
                        index_tweet(prepared_tweet)
                        print("Creating Original tweet")
                 
                else:
                    print(f"abnormal tweet ID {tweet_id}")
                    continue



            else:

                #check if user is there or not
                user_id=user_info.get("id_str")
                if es.exists(index=index_name_userids, id=user_id):
                    print(f"User ID {user_id} already exists.")
                else:
                    es.index(index=index_name_userids, id=user_id, document={"user_id": user_id})
                    insert_user_mysql(user_info)
                    connection.commit()
                    print("commit success")
                    #if not load user into user index of es
                    # load or update user info in mysql

                #load info into new table, tweetid, userid,screen_name, text, creationtime, retweet_count, Fav_count, retweeted, favourited, 
                #isretweet=false, Rtweetid, Ruserid,screen_name, Rcreationtime, retweet_count, Fav_count, retweeted, favourited

                #load info into main table
                isretweet=False
                prepared_tweet=preparedTweet(tweet,user_info,retweeted_status,retweeted_user_info,isretweet)
                index_tweet(prepared_tweet)





            #es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id, "user_id": user_id})

connection.close()
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")
print("All tweets have been indexed.")

insert successfull
commit success
insert successfull
commit success
Creating Original tweet
insert successfull
commit success
insert successfull
commit success
Creating Original tweet
insert successfull
commit success
insert successfull
commit success
Creating Original tweet
insert successfull
commit success
insert successfull
commit success
Creating Original tweet
insert successfull
commit success
insert successfull
commit success
Creating Original tweet
insert successfull
commit success
insert successfull
commit success


IntegrityError: (1062, "Duplicate entry '14135350' for key 'users1.PRIMARY'")

In [30]:
prepared_tweet

{'created_at': 'Sun Apr 12 18:27:26 +0000 2020',
 'id': 1249403770435493888,
 'id_str': '1249403770435493888',
 'text': 'RT @biannagolodryga: This image.\nThis quote.\n\n“One of the reasons the pandemic is hitting the country so hard is that no one appears to be…',
 'user': {'id': 4707764075,
  'id_str': '4707764075',
  'name': 'UpsidedownTurtle 🧢',
  'screen_name': 'Adakisn'},
 'retweeted_status': {'created_at': 'Sun Apr 12 12:40:06 +0000 2020',
  'id': 1249316363681910784,
  'id_str': '1249316363681910784',
  'text': 'This image.\nThis quote.\n\n“One of the reasons the pandemic is hitting the country so hard is that no one appears to… https://t.co/fypNKXxlpb',
  'user': {'id': 14135350,
   'id_str': '14135350',
   'name': 'Bianna Golodryga',
   'screen_name': 'biannagolodryga'},
  'retweet_count': 3085,
  'favorite_count': 6343,
  'favorited': False,
  'retweeted': False,
  'possibly_sensitive': False},
 'retweet_count': 0,
 'favorite_count': 0,
 'favorited': False,
 'retweeted': Fal

In [6]:
import json

# Open and read the file
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", 'r') as file:

    line_number = 0
    for line in file:
        line_number += 1
        # Skip empty lines or lines that do not contain valid JSON
        if not line.strip():
            continue
        try:
            tweet = json.loads(line)
            # Use tweet ID as the document ID in Elasticsearch
            res = es.index(index=index_name, id=tweet["id"], document=tweet)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON at line {line_number}: {e}")


In [71]:
def search_tweets(query_string):
    query = {
        "query": {
            "match": {
                "text": query_string
            }
        },
        "_source": ["created_at", "user.name", "text"]  # Specify the fields you want to retrieve
    }
    results = es.search(index=index_name, body=query)
    for hit in results['hits']['hits']:
        # Extracting and printing the specified fields from each tweet
        tweet_data = hit['_source']
        print(f"Username: {tweet_data['user']['name']}, Created At: {tweet_data['created_at']}, Text: {tweet_data['text']}")

# Example search
start_time= time.time()
search_tweets("#football")
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")

Username: Steve Brookes MBE, Created At: Sun Apr 12 18:36:02 +0000 2020, Text: #FOOTBALL 'Coronavirus is God's yellow card to the world' - Sredojevic https://t.co/YPJ5hrYHeC
Username: 🎙, Created At: Sun Apr 12 18:41:18 +0000 2020, Text: Corona has taken away 36 days of Football and Messi off the pitch.

Despite that guys #StayAtHomeSaveLives

Wishing everyone #HappyEaster
Username: Zaddysalam, Created At: Sun Apr 12 18:28:55 +0000 2020, Text: If we have to sacrifice one football club to end corona virus, which club will you pick? I will pick *arsenal*😂
Username: Partha Ghosh, Created At: Sun Apr 12 18:29:39 +0000 2020, Text: My knowledge of Uruguay is about #football and @LuisSuarez9's famous fouls 😬 ... this is not about the country, but… https://t.co/uZLBDhhfv4
Total runtime of the program is 0.039427995681762695 seconds


In [74]:
def search_tweets(query_string):
    query = {
        "query": {
            "match": {
                "text": query_string
            }
        },
        "_source": ["created_at", "user.name", "text"]  # Specify the fields you want to retrieve
    }
    results = es.search(index=index_name, body=query)
    for hit in results['hits']['hits']:
        # Extracting and printing the specified fields from each tweet
        tweet_data = hit['_source']
        print(f"Username: {tweet_data['user']['name']}, Created At: {tweet_data['created_at']}, Text: {tweet_data['text']}")

# Example search
start_time= time.time()
search_tweets("cricket")
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")

Username: Today News Breaking, Created At: Sun Apr 12 18:29:21 +0000 2020, Text: Corona virus can curb Liam Plunkett’s cricket career https://t.co/X88GGdzFZr https://t.co/eYiiqZKmGn
Total runtime of the program is 0.04018592834472656 seconds


In [103]:
def search_tweets(query_string):
    # Building a query that includes a condition on the `isretweet` field
    
    query = {
        "query": {
            "bool": {
                "must": {
                    "match": {
                        "text": query_string
                    }
                },
                "filter": {
                    "term": {
                        "isretweet": False
                    }
                }
            }
        },
        "_source": ["created_at", "user.name", "text", "id_str", "retweet_count", "favorite_count"]  # Adding fields to be retrieved
    }
    
    # Execute the search query
    results = es.search(index=index_name, body=query)
    print("Search Results:\n" + "-"*80)  # Print a header and a separator
    for hit in results['hits']['hits']:
        # Extracting the specified fields from each tweet
        tweet_data = hit['_source']
        print(f"Tweet ID: {tweet_data.get('id_str')}")
        print(f"Username: {tweet_data['user']['name']}")
        print(f"Created At: {tweet_data['created_at']}")
        print(f"Text: {tweet_data['text']}")
        print(f"Retweets: {tweet_data.get('retweet_count', 0)}")
        print(f"Likes: {tweet_data.get('favorite_count', 0)}")
        print("-"*80)  # Print a line after each tweet's details

# Example usage
import time
start_time = time.time()
search_tweets("football")
end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")


Search Results:
--------------------------------------------------------------------------------
Tweet ID: 1249405936910901248
Username: Steve Brookes MBE
Created At: Sun Apr 12 18:36:02 +0000 2020
Text: #FOOTBALL 'Coronavirus is God's yellow card to the world' - Sredojevic https://t.co/YPJ5hrYHeC
Retweets: 0
Likes: 0
--------------------------------------------------------------------------------
Tweet ID: 1249407261539045376
Username: 🎙
Created At: Sun Apr 12 18:41:18 +0000 2020
Text: Corona has taken away 36 days of Football and Messi off the pitch.

Despite that guys #StayAtHomeSaveLives

Wishing everyone #HappyEaster
Retweets: 0
Likes: 0
--------------------------------------------------------------------------------
Tweet ID: 1249404143254540288
Username: Zaddysalam
Created At: Sun Apr 12 18:28:55 +0000 2020
Text: If we have to sacrifice one football club to end corona virus, which club will you pick? I will pick *arsenal*😂
Retweets: 0
Likes: 0
-----------------------------------

In [97]:
def search_user_tweets(user_id):
    query = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"user.id_str": user_id}}  # Filter to match the user ID
                ]
            }
        },
        "sort": [
            {"created_at": {"order": "asc"}}  # Sort the tweets by creation date in ascending order
        ],
        "_source": ["created_at", "user.name", "text", "isretweet"]  # Specify the fields you want to retrieve
    }
    
    # Execute the search query
    results = es.search(index=index_name, body=query)#, size=1000)  # Adjust size as needed
    print("Search Results:\n" + "-"*80)  # Print a header and a separator
    for hit in results['hits']['hits']:
        # Extracting the specified fields from each tweet
        tweet_data = hit['_source']
        print(f"Tweet ID: {hit['_id']}")  # Using the document ID as Tweet ID
        print(f"Username: {tweet_data['user']['name']}")
        print(f"Created At: {tweet_data['created_at']}")
        print(f"Text: {tweet_data['text']}")
        print(f"Is Retweet: {'Yes' if tweet_data.get('isretweet', False) else 'No'}")
        print("-"*80)  # Print a line after each tweet's details

# Example usage
user_id = "1078419185968074752"
search_user_tweets(user_id)


Search Results:
--------------------------------------------------------------------------------
Tweet ID: 1249346683735085063
Username: ❤Lucias news 📰 Cafe @ home❤
Created At: Sun Apr 12 14:40:35 +0000 2020
Text: ⚠️ leseempfehlung ⚠️
Corona!!!
Bitte🔃🔃🔃🔃🔃
retweeten ‼‼‼
#antifaforhelp
Is Retweet: No
--------------------------------------------------------------------------------
Tweet ID: 1249396923628486661
Username: ❤Lucias news 📰 Cafe @ home❤
Created At: Sun Apr 12 18:00:13 +0000 2020
Text: ⚠️ leseempfehlung ⚠️
Corona!!!
Bitte🔃🔃🔃🔃🔃
retweeten ‼‼‼
#antifaforhelp
Is Retweet: No
--------------------------------------------------------------------------------
Tweet ID: 1249397251027472389
Username: ❤Lucias news 📰 Cafe @ home❤
Created At: Sun Apr 12 18:01:31 +0000 2020
Text: ⚠️ leseempfehlung ⚠️
Corona!!!
Bitte🔃🔃🔃🔃🔃
retweeten ‼‼‼
#antifaforhelp
Is Retweet: No
--------------------------------------------------------------------------------
Tweet ID: 1249397764703862794
Username: ❤Lucias new

In [84]:
query = {
        "query": {
            "match": {
                "text": "corona"
            }
        },
        "_source": ["created_at", "user.name", "text"]  # Specify the fields you want to retrieve
    }
results = es.search(index=index_name, body=query)
results = es.search(index=index_name)

In [20]:
    for hit in results['hits']['hits']:
        # Extracting and printing the specified fields from each tweet
        tweet_data = hit['_source']
        break

In [21]:
tweet_data

{'created_at': 'Sun Apr 12 18:30:25 +0000 2020',
 'id': 1249404522662895618,
 'id_str': '1249404522662895618',
 'text': 'RT @ZeyniAKD1453: Pensilvanyada vaka gid gide cogaliyor Senin Radarinin Antenine Tukureyim Corona Ara Bul Şu Orospu cocugunu sana guveniyor…',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1229865631463485440,
  'id_str': '1229865631463485440',
  'name': 'İbrahim Aycıl',
  'screen_name': 'brahimAycl2',
  'location': None,
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1206,
  'friends_count': 1826,
  'listed_count': 0,
  'favourites_count': 12412,
  'statuses_count': 5830,
  'created_at': 'Tue Feb 18 20:30:09 +0000 2020',
  'utc_offs

In [2]:
# this code block allows you to inspect 2 tweets and 2 retweets in the dataset
# it also saves all the unique tweets/retweets in the dictionary tweets
# this code will help you to understand the data
# feel free to print out more fields

lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
                #print out some fields of one retweet 
                #note that you should look at other fields too
                if (print_retweet < 2):
                    print_retweet += 1
                    print('RETWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    
                    print('User-- ',data['user'],'\n')
                  
                    print('retweetstatus--',data['retweeted_status'],'\n')
                  

            else:
                num_tweets += 1
                #print out some fields of one tweet 
                #note that you should look at other fields too
                if (print_tweet < 2):
                    print_tweet += 1
                    print('TWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    print('User-- ',data['user'],'\n')
                    


            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue
            
print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

RETWEET
 id-- 1249403767180668930 text-- RT @nuffsaidny: wishing death on people is weirdo behavior. 

User--  {'id': 1242817830946508801, 'id_str': '1242817830946508801', 'name': 'juwelz v', 'screen_name': 'juwelz_v', 'location': 'Lower East Side, Manhattan', 'url': 'https://linktr.ee/juwelzv', 'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 43, 'friends_count': 118, 'listed_count': 0, 'favourites_count': 722, 'statuses_count': 906, 'created_at': 'Wed Mar 25 14:17:28 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile

num of lines= 18518 num of tweets= 7356 num of retweets= 11162
num of unique tweets/retweets= 18506


In [14]:
# print the last data item
data

{'created_at': 'Sun Apr 12 18:47:53 +0000 2020',
 'id': 1249408918196871168,
 'id_str': '1249408918196871168',
 'text': 'RT @liputan6dotcom: Lagi, 2 Dokter Senior Meninggal karena Corona Covid-19 https://t.co/eDbMsxFA84',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 2283278160,
  'id_str': '2283278160',
  'name': 'Mr rius',
  'screen_name': 'AwhexWibowo',
  'location': 'Yogyakarta, Indonesia',
  'url': None,
  'description': 'have fun go mad',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 451,
  'friends_count': 608,
  'listed_count': 2,
  'favourites_count': 4625,
  'statuses_count': 31160,
  'created_at': 'Thu Jan 09 08:34:03 +0000 2014',
  'utc_offset': None,
  'time_zone': None,
  'g

In [15]:
data['user']

{'id': 2283278160,
 'id_str': '2283278160',
 'name': 'Mr rius',
 'screen_name': 'AwhexWibowo',
 'location': 'Yogyakarta, Indonesia',
 'url': None,
 'description': 'have fun go mad',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 451,
 'friends_count': 608,
 'listed_count': 2,
 'favourites_count': 4625,
 'statuses_count': 31160,
 'created_at': 'Thu Jan 09 08:34:03 +0000 2014',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'pr

In [16]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # pseudocode: 
            # if tweet has been seen before, 
            #         continue (ignore this tweet, go to next iteration of loop)
            
            user = data['user']
            # psuedocode: 
            # if user has not been seen before
            #            add to user table 
            # update datastores with any metrics that you are tracking 
            
            if ( data['text'].startswith('RT') ):
                # psuedocode:
                # update retweet information
                # note that you may not have an entry for the original tweet 
                # if that is not in the dataset
                pass  # does nothing right now
                
            else:
                # psuedocode:
                # add the new tweet to datastore
                pass # does nothing right now
        except:
            continue


In [ ]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # pseudocode: 
            # if tweet has been seen before, 
            #         continue (ignore this tweet, go to next iteration of loop)
            
            user = data['user']
            # psuedocode: 
            # if user has not been seen before
            #            add to user table 
            # update datastores with any metrics that you are tracking 
            
            if ( data['text'].startswith('RT') ):
                # psuedocode:
                # update retweet information
                # note that you may not have an entry for the original tweet 
                # if that is not in the dataset
                pass  # does nothing right now
                
            else:
                # psuedocode:
                # add the new tweet to datastore
                pass # does nothing right now
        except:
            continue
